In [173]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Read the data file and convert it into a pandas data frame 
df=pd.read_json('transaction-data-adhoc-analysis.json') 

#Cleaning the Data Frame

df.drop(['address','birthdate','mail','sex','username','transaction_value'],axis=1,inplace=True)

#Transforming the data into one item per row 
df['transaction_items'] = df['transaction_items'].apply(lambda x: x.split(';'))
df=df.explode('transaction_items')

#Creating a new quantity column 
def quantity_column(row):
    if row['transaction_items'][-1] == ")":
        return row['transaction_items'][-2]
    else:
        return 1
df['Quantity'] = df.apply(lambda row: quantity_column(row), axis=1)

#Removing the quantity in transaction_items
def clean_transaction_items(row):
    if row['transaction_items'][-1] == ")":
        return row['transaction_items'][:-5]
    else:
        return row['transaction_items']
df['transaction_items'] = df.apply(lambda row: clean_transaction_items(row), axis=1)

#Removing the Brand name from the transaction_items
df[['Brand', 'Item']] = df['transaction_items'].str.split(',', expand=True)
df.drop(['transaction_items','Brand'],axis=1,inplace=True)


#Convert the dates into month number
df['transaction_date'] = pd.to_datetime(df['transaction_date']) 
df['Month'] = pd.DatetimeIndex(df['transaction_date']).month
df.drop(['transaction_date'],axis=1,inplace=True)
df=df[['name','Item','Quantity','Month']]
df

,name,Item,Quantity,Month
0,Jennifer Campbell,Beef Chicharon,4,1
1,Melissa Kim,Beef Chicharon,3,1
1,Melissa Kim,Nutrional Milk,4,1
1,Melissa Kim,Orange Beans,1,1
2,Melissa Kim,Gummy Vitamins,3,1
...,...,...,...,...
83034,Donald Andersen,Nutrional Milk,2,6
83034,Donald Andersen,Gummy Worms,2,6
83035,Stephanie Russell,Nutrional Milk,1,6
83035,Stephanie Russell,Gummy Worms,3,6


In [144]:
def quantitysold(month):
    countdf=df.copy()
    countdf.drop(['name'],axis=1,inplace=True)
    
    #Filter by month
    countdf=countdf[countdf.Month==month]
    #Group by Items and Sum 
    countdf['Quantity']=countdf['Quantity'].astype('int')
    countdf=countdf.groupby(['Item'],as_index=False)['Quantity'].sum()
    
    return countdf

In [152]:
quantitysold(6)

,Item,Quantity
0,Beef Chicharon,9902
1,Gummy Vitamins,9980
2,Gummy Worms,9934
3,Kimchi and Seaweed,10104
4,Nutrional Milk,9767
5,Orange Beans,10106
6,Yummy Vegetables,9722


In [142]:
def salevalue(month):
    saledf=quantitysold(month)
    
    #Adding the prices for each items 
    saledf['Price'] = [1299,1500,150,799,1990,199,500] 
    
    #Solving for the sale value by multiplying the quantity sold of each it
    saledf['Sale_Value']=saledf['Quantity']*saledf['Price']
    return saledf

In [158]:
salevalue(6)

,Item,Quantity,Price,Sale_Value
0,Beef Chicharon,9902,1299,12862698
1,Gummy Vitamins,9980,1500,14970000
2,Gummy Worms,9934,150,1490100
3,Kimchi and Seaweed,10104,799,8073096
4,Nutrional Milk,9767,500,4883500
5,Orange Beans,10106,1990,20110940
6,Yummy Vegetables,9722,199,1934678


In [254]:
def repeat_users(month):
    repeatdf=df.copy()
    
    #Creating a dataframe for the previous month and the current month
    currentdf=repeatdf[repeatdf.Month==month]
    prevdf=repeatdf[repeatdf.Month==month-1]
    
    if month>1:
        #Getting the list of unique names for the previous month and the current month
        prevmonth_list = prevdf.name.unique()
        currentmonth_list=currentdf.name.unique()
        repeatcount=0
        
        #If the name is in the previous month and the current month, then it will add to the repeat counter
        for name in currentmonth_list:
            if name in prevmonth_list:
                repeatcount +=1
        return repeatcount
    else:
        return 0

In [271]:
def inactive_users(month):
    inactivedf=df.copy()
    
    #Creating a data frame for the previous months and the current month
    pastdf=inactivedf[inactivedf.Month<month]
    currentdf=inactivedf[inactivedf.Month==month]
    
    if month>1:
        #Getting the list of unique names for the previous months and the current month
        past_list=pastdf.name.unique()
        currentmonth_list=currentdf.name.unique()
        inactivecount=0
        
        #If the name is in the previous months but not in the current month, then it will add the the inactive counter
        for name in past_list:
            if name not in currentmonth_list:
                inactivecount+=1
        return inactivecount
        
    else:
        return 0

In [315]:
def engaged_users(month):
    engageddf=df.copy()
    
    #Getting the dataframe for each month
    jandf=engageddf[engageddf.Month==1]
    febdf=engageddf[engageddf.Month==2]
    marchdf=engageddf[engageddf.Month==3]
    aprildf=engageddf[engageddf.Month==4]
    maydf=engageddf[engageddf.Month==5]
    junedf=engageddf[engageddf.Month==6]
    
    #Getting the list of unique names for each month
    jan_list=jandf.name.unique()
    feb_list=febdf.name.unique()
    march_list=marchdf.name.unique()
    april_list=aprildf.name.unique()
    may_list=maydf.name.unique()
    june_list=junedf.name.unique()
    
    #Getting the dataframe list of unique name for the current month
    currentdf=engageddf[engageddf.Month==month]
    currentmonth_list=currentdf.name.unique()
    engagedcount=0

    #Counting the number of people who purchased consistently and this current month
    for name in currentmonth_list:
        if month==1:
            if name in jan_list:
                engagedcount+=1
        elif month==2:
            if name in feb_list:
                engagedcount+=1
        elif month==3:
            if name in march_list:
                engagedcount+=1
        elif month==4:
             if name in april_list:
                engagedcount+=1
        elif month==5:
            if name in may_list:
                engagedcount+=1
        elif month==6:
            if name in june_list:
                engagedcount+=1
    return engagedcount 

In [327]:
#Creating a new dataframe to aggegate customer frequency data
#Create columns for each months
January=[repeat_users(1),inactive_users(1),engaged_users(1)]
February=[repeat_users(2),inactive_users(2),engaged_users(2)]
March=[repeat_users(3),inactive_users(3),engaged_users(3)]
April=[repeat_users(4),inactive_users(4),engaged_users(4)]
May=[repeat_users(5),inactive_users(5),engaged_users(5)]
June=[repeat_users(6),inactive_users(6),engaged_users(6)]

list_of_months = list(zip(January, February, March, April, May, June))
membershipdf = pd.DataFrame(list_of_months,columns=['January', 'February', 'March', 'April', 'May', 'June'])

#Set index Repeaters, Inactive, Engaged
index = pd.Index(['Repeaters', 'Inactive', 'Engaged'])
membershipdf = membershipdf.set_index([index])
membershipdf

,January,February,March,April,May,June
Repeaters,0,5172,5216,5154,5110,5193
Inactive,0,1416,1747,1909,1917,1835
Engaged,6588,6631,6622,6556,6568,6652


In [276]:
#Additional Metric#1 Top 3 customers in the past 6 months
topcustomersdf=df.copy()
topcustomersdf.drop(['Quantity','Item'],axis=1,inplace=True)
topcustomersdf.name.value_counts().head(3)

Michael Johnson        126
David Smith            120
Christopher Johnson    112
Name: name, dtype: int64

In [277]:
#Additional Metric#2 Best Selling Items in the past 6 months
topsellingdf=df.copy()
topsellingdf['Quantity']=topsellingdf['Quantity'].astype('int')
topsellingdf=topsellingdf.groupby(['Item'],as_index=False)['Quantity'].sum()
topsellingdf=topsellingdf.sort_values(by='Quantity', ascending=False)
topsellingdf.head(5)

,Item,Quantity
1,Gummy Vitamins,59576
6,Yummy Vegetables,59429
5,Orange Beans,59406
3,Kimchi and Seaweed,59390
2,Gummy Worms,59319


In [278]:
#Additional Metric#3 Top Income-generating items in the past 6 months
topearningdf=topsellingdf.copy()
topearningdf['Price'] = [1500,500,199,799,150,1299,1990] 
topearningdf['Sale_Value']=topearningdf['Quantity']*topearningdf['Price']
topearningdf=topearningdf.sort_values(by='Sale_Value', ascending=False)
topearningdf.head(5)

,Item,Quantity,Price,Sale_Value
4,Nutrional Milk,58728,1990,116868720
1,Gummy Vitamins,59576,1500,89364000
0,Beef Chicharon,59302,1299,77033298
3,Kimchi and Seaweed,59390,799,47452610
6,Yummy Vegetables,59429,500,29714500
